In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import pymysql


from sqlalchemy import create_engine
from tqdm import tqdm
from binance import Client 
sns.set()
pymysql.install_as_MySQLdb()


In [3]:
engine = create_engine('sqlite:///Binance_daily.db')

In [4]:
client = Client()

In [5]:
symbols = ['BTCUSDT','ETHUSDT', 'ADAUSDT', 'BNBUSDT']

In [6]:
def get_data(symbol, interval = client.KLINE_INTERVAL_12HOUR, start = '2018-06-01'):
    temp = client.get_historical_klines(symbol, interval, start)
    temp = pd.DataFrame(temp)
    temp = temp.iloc[:,:6]
    temp.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
    temp.Date = pd.to_datetime(temp.Date, unit = "ms")
    temp = temp.set_index("Date", drop = True)
    temp = temp.astype(float)
    return temp

In [9]:
for coin in tqdm(symbols):
    df = get_data(coin)
    df.to_sql(coin, if_exists= 'replace', con = engine)

100%|██████████| 4/4 [00:28<00:00,  7.00s/it]


In [10]:
def qry(symbol):
    df = pd.read_sql(symbol, engine)
    df = df.set_index('Date')
    df = df.asfreq('12H')
    return df.Close

In [23]:
def apply_techinical(df):
    
    for column in df.columns.to_list():
        #calculating returns
        df[f'Ret_{column}'] = df[column].pct_change().mul(100)
        df[f'Ret_cum_{column}'] = df[f'Ret_{column}'].cumsum()
        #normalizing values
        benchmark = df.loc[:,column][0]
        df[f'Norm_{column}'] = df[column].div(benchmark).mul(100)
    return df
        

In [24]:
def process_coin(symbols):
    df_list = []
    
    for coin in symbols:
        df = qry(coin)
        df.name = coin 
        df_list.append(df)
    df = pd.DataFrame(df_list).T
    df = apply_techinical(df)
    
    return df

In [25]:
df = process_coin(symbols = symbols)

In [26]:
df

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT,Ret_BTCUSDT,Ret_cum_BTCUSDT,Norm_BTCUSDT,Ret_ETHUSDT,Ret_cum_ETHUSDT,Norm_ETHUSDT,Ret_ADAUSDT,Ret_cum_ADAUSDT,Norm_ADAUSDT,Ret_BNBUSDT,Ret_cum_BNBUSDT,Norm_BNBUSDT
Date,,,,,,,,,,,,,,,,
2018-06-01 00:00:00,7361.39,570.63,0.21580,13.7800,NaN,NaN,100.000000,NaN,NaN,100.000000,NaN,NaN,100.000000,NaN,NaN,100.000000
2018-06-01 12:00:00,7521.01,579.00,0.22038,14.2888,2.168340,2.168340,102.168340,1.466800,1.466800,101.466800,2.122335,2.122335,102.122335,3.692308,3.692308,103.692308
2018-06-02 00:00:00,7652.28,592.73,0.22428,14.5000,1.745377,3.913718,103.951563,2.371330,3.838130,103.872912,1.769671,3.892006,103.929564,1.478081,5.170388,105.224964
2018-06-02 12:00:00,7640.03,590.85,0.22648,14.6732,-0.160083,3.753635,103.785155,-0.317176,3.520953,103.543452,0.980917,4.872923,104.949027,1.194483,6.364871,106.481858
2018-06-03 00:00:00,7714.85,619.93,0.23284,14.7861,0.979316,4.732950,104.801539,4.921723,8.442676,108.639574,2.808195,7.681118,107.896200,0.769430,7.134301,107.301161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-25 00:00:00,30716.03,1918.37,0.29810,240.2000,0.617805,262.724610,417.258561,2.315249,321.074468,336.184568,3.077455,305.644200,138.137164,1.607445,506.181376,1743.105951
2023-06-25 12:00:00,30462.66,1898.80,0.29210,238.8000,-0.824879,261.899732,413.816684,-1.020137,320.054331,332.755025,-2.012747,303.631452,135.356812,-0.582848,505.598528,1732.946299
2023-06-26 00:00:00,30322.89,1880.87,0.28900,237.9000,-0.458824,261.440908,411.917994,-0.944281,319.110050,329.612884,-1.061280,302.570172,133.920297,-0.376884,505.221644,1726.415094


In [27]:
def train_split(df, size:int):
    percentage = int(len(df) * size)
    data = df.iloc[:percentage]
    data_test = df.iloc[percentage:]
    
    data.to_csv('../data/processed/data_processed.csv')
    data_test.to_csv('../data/processed/data_test.csv')
    
    return len(data), len(data_test)

In [28]:
train_split(df, 0.7)

(2593, 1112)